In [1]:
import pandas as pd
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
#Data import

train_data_file_name = "loans_train_data.csv"
test_data_file_name = "loans_test_data.csv"
raw_data_df_train = pd.read_csv(train_data_file_name)
raw_data_df_test = pd.read_csv(test_data_file_name)

raw_data_df_train

,emp_title,emp_length,state,homeownership,annual_income,verified_income,debt_to_income,annual_income_joint,verification_income_joint,debt_to_income_joint,...,tax_liens,public_record_bankrupt,loan_purpose,application_type,loan_amount,term,interest_rate,initial_listing_status,disbursement_method,id
0,instructor,2.0,NY,RENT,45000.0,Source Verified,6.71,NaN,NaN,NaN,...,0,0,debt_consolidation,individual,12000,60,19.03,whole,Cash,1
1,owner,10.0,CA,RENT,68000.0,Source Verified,18.99,NaN,NaN,NaN,...,0,1,small_business,individual,7200,36,16.01,whole,Cash,2
2,merchandiser,10.0,TN,OWN,15000.0,Source Verified,20.83,NaN,NaN,NaN,...,0,0,credit_card,individual,5000,36,11.99,whole,Cash,3
3,deputy clerk,2.0,OH,MORTGAGE,35000.0,Verified,35.56,NaN,NaN,NaN,...,0,0,debt_consolidation,individual,12800,60,23.88,whole,DirectPay,4
4,teacher,2.0,UT,RENT,34000.0,Not Verified,13.20,NaN,NaN,NaN,...,0,0,credit_card,individual,1200,36,10.90,whole,Cash,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,human capital consultant,1.0,VA,MORTGAGE,115000.0,Not Verified,22.79,NaN,NaN,NaN,...,0,0,credit_card,individual,20000,36,7.34,whole,Cash,6996
6996,director of property operations,3.0,CA,RENT,116480.0,Verified,13.74,NaN,NaN,NaN,...,0,0,credit_card,individual,10000,36,10.42,whole,Cash,6997
6997,rn,9.0,CA,RENT,200000.0,Source Verified,8.38,NaN,NaN,NaN,...,0,0,credit_card,individual,16000,36,9.93,whole,Cash,6998
6998,attorney - member,8.0,VT,MORTGAGE,350000.0,Source Verified,11.98,NaN,NaN,NaN,...,0,0,credit_card,individual,15000,36,12.61,whole,Cash,6999


In [3]:
# Very basic data cleaning

percent_thres = 0.5
thresh = percent_thres*raw_data_df_train.shape[0]
thresh


filtered_df_train = raw_data_df_train.dropna(axis=1, thresh=thresh)

dropped_cols = set(raw_data_df_train.columns) - set(filtered_df_train.columns)

filtered_df_test = raw_data_df_test.drop(columns=dropped_cols)
filtered_df_train.head()


,emp_title,emp_length,state,homeownership,annual_income,verified_income,debt_to_income,delinq_2y,earliest_credit_line,inquiries_last_12m,...,tax_liens,public_record_bankrupt,loan_purpose,application_type,loan_amount,term,interest_rate,initial_listing_status,disbursement_method,id
0,instructor,2.0,NY,RENT,45000.0,Source Verified,6.71,0,2011,0,...,0,0,debt_consolidation,individual,12000,60,19.03,whole,Cash,1
1,owner,10.0,CA,RENT,68000.0,Source Verified,18.99,0,1971,1,...,0,1,small_business,individual,7200,36,16.01,whole,Cash,2
2,merchandiser,10.0,TN,OWN,15000.0,Source Verified,20.83,0,2007,1,...,0,0,credit_card,individual,5000,36,11.99,whole,Cash,3
3,deputy clerk,2.0,OH,MORTGAGE,35000.0,Verified,35.56,0,2007,3,...,0,0,debt_consolidation,individual,12800,60,23.88,whole,DirectPay,4
4,teacher,2.0,UT,RENT,34000.0,Not Verified,13.20,0,2011,0,...,0,0,credit_card,individual,1200,36,10.90,whole,Cash,5


In [4]:
#Temporary!!!
filtered_df_train = filtered_df_train.drop(columns=["emp_title"])
filtered_df_train = filtered_df_train.dropna(axis=0)

filtered_df_test = filtered_df_test.drop(columns=["emp_title"])
filtered_df_test = filtered_df_test.dropna(axis=0)

In [8]:
from sklearn.linear_model import LinearRegression, Ridge

# Find catagorical column names
num_cols = filtered_df_train._get_numeric_data().columns
cat_cols = list(set(filtered_df_train.columns) - set(num_cols))
cat_cols.append("emp_length")


def model(train_df, test_df, catagorical_cols):
    # Combine train and test data and keep track of idx needed to separate them
    train_len = len(filtered_df_train.index)
    test_len = len(filtered_df_test.index)
    cutoff_idx = train_len
    combined_df = pd.concat([filtered_df_train, filtered_df_test])

    # One hot encode
    combined_df_ohe = pd.get_dummies(combined_df, columns=cat_cols)    

    # Separate train and test again
    train_df_ohe = combined_df_ohe[0:cutoff_idx]
    test_df_ohe = combined_df_ohe[cutoff_idx:]
    test_df_ohe = test_df_ohe.drop(columns=["interest_rate"]) # Test data doesn't need this

    # Do train-test split to get train and validation test set
    X_df = train_df_ohe.drop(columns=["interest_rate"])
    y_df = train_df_ohe["interest_rate"]
    X_train, X_validate, y_train, y_validate = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

    # Fit model
    regr = LinearRegression()
    regr.fit(X_train, y_train)
    
    # Return score
    return regr.score(X_validate, y_validate)

In [9]:
model(filtered_df_train, filtered_df_test, cat_cols)

0.365436803665081